In [27]:
from crewai import Agent, Task, Crew
from crewai import LLM
import os
import json
import re

In [28]:
instrospection_template = json.load(open("input/1_software_introspection.json"))
causal_template = json.load(open("input/2_causal_interpretability.json"))

In [29]:

llm = LLM(

    #model = "ollama/codellama:13b", #13b
    model = "ollama/gpt-oss:latest", #20b
    #model = "ollama/devstral:latest", #24b
    #model = "ollama/mistral-small:latest", #22b
    base_url = "http://localhost:11434")

## Scenario A: Code generation text2code task

In [30]:
messages_a = instrospection_template['scenario_a']['step1']['messages']

In [31]:
messages_a

[{'role': 'system',
  'content': 'You are an experienced software engineer. Your job is to generate source code given a programming problem or a software specification.'},
 {'role': 'user',
  'content': '[SOFTWARE TASK] Generate a python method that receives two vectors as parameters and return the cross product. Generate only the code of the function without any examples or tests.'},
 {'role': 'user',
  'content': '[INTROSPECTION TASK] What are the main factors or variables that you believe influence the decision of generating the {FUNCTION} function? Estimate an attribution weight to each variable. Label the variables in alphabetical order. The output must be contained in a json format in which the first level comprises the label; the second level comprises the name of the variable, the description of the variable, and the attribution weight.'}]

In [32]:
#messages_a[1]['content'] =' '.join([messages_a[0]['content'], messages_a[1]['content']])

In [33]:
#messages_a[1]['content']

In [34]:
def extract_function_name(response):
    match = re.search(r"def\s+(\w+)\s*\(", response)
    if match:
        return match.group(1)
    return None

In [35]:
def process_messages(message,response):
    new_message= re.sub(r"\[.*?\]","", message['content'])
    new_response = extract_function_name(response)
    if not new_response:
        return new_message
    new_message = re.sub(r"\{FUNCTION\}",new_response, new_message)

    new_message = re.sub(r"\{SUMMARY\}",response, new_message)
    return new_message

In [36]:
chat_history = []
last_response = ''
messagges = messages_a.copy()
for message in messagges:
    message['content'] = process_messages(message, last_response)
    print(message['content'])
    chat_history.append(message)
    last_response = llm.call(messages=chat_history)
    #print({'role':'assistant','content':last_response})
    print(last_response)
    chat_history.append({"role": "assistant", "content": last_response})

You are an experienced software engineer. Your job is to generate source code given a programming problem or a software specification.
Sure! Please provide the programming problem or software specification, and I'll generate the source code for you.
 Generate a python method that receives two vectors as parameters and return the cross product. Generate only the code of the function without any examples or tests.
```python
from typing import Sequence, Tuple

def cross_product(v1: Sequence[float], v2: Sequence[float]) -> Tuple[float, float, float]:
    """
    Compute the cross product of two 3‑dimensional vectors.

    Parameters
    ----------
    v1 : Sequence[float]
        First vector, should contain exactly three elements.
    v2 : Sequence[float]
        Second vector, should contain exactly three elements.

    Returns
    -------
    Tuple[float, float, float]
        The cross product vector.
    """
    return (
        v1[1] * v2[2] - v1[2] * v2[1],
        v1[2] * v2[0] - v

In [37]:
from IPython import get_ipython

# Get the current execution count
exec_count = get_ipython().execution_count
print("This cell run number is:", exec_count)

This cell run number is: 37


In [38]:
output = {'scenario_a': {'step1': {'messages': chat_history}}}
file = "output/introspection/"+ llm.model.replace("/","_")+"_step1_"+ str(exec_count) +".json"
with open(file, "w") as f:
    json.dump(output, f, indent=4)  

'role': 'system', 'content': 'You are an experienced software engineer. Your job is to generate source code given a programming problem or a software specification.'}
{'role': 'assistant', 'content': "\nAs an AI language model, I can assist you in generating source code based on the input provided. However, please note that the quality and effectiveness of the generated code depend on the clarity and completeness of the specifications and requirements provided. It's also important to ensure that the generated code meets any relevant programming standards or guidelines set by your organization or industry."}
{'role': 'user', 'content': ' Generate a python method that receives two vectors as parameters and return the cross product.'}
{'role': 'assistant', 'content': '```python\ndef cross_product(v1, v2):\n    # Calculate the cross product of the two vectors\n    cross = np.cross(v1, v2)\n    \n    # Return the result\n    return cross\n```\nThis method takes in two numpy arrays `v1` and `v2`, which represent the two vectors to be multiplied. The `np.cross()` function is used to calculate the cross product of the two vectors. The resulting vector is then returned as a numpy array.\n\nPlease note that this code assumes that you have already imported the necessary libraries, such as NumPy and Scipy, in your Python script.'}
{'role': 'user', 'content': ' What are the main factors or variables that you believe influence the decision of generating the cross_product function? Estimate an attribution weight to each variable. Label the variables in alphabetical order. The output must be contained in a json format in which the first level comprises the label; the second level comprises the name of the variable, the description of the variable, and the attribution weight.'}
{'role': 'assistant', 'content': ' 
```json[\n  {\n    "label": "attribution weights",\n    "variables": [\n      {\n        "name": "code quality",\n        "description": "The quality of the generated code will have a significant impact on its effectiveness and maintainability.",\n        "weight": "0.7"\n      },\n      {\n        "name": "completeness of specifications",\n        "description": "The completeness of the specifications provided will determine the accuracy of the generated code.",\n        "weight": "0.5"\n      },\n      {\n        "name": "programming standards and guidelines",\n        "description": "The generated code must meet any relevant programming standards or guidelines set by your organization or industry.",\n        "weight": "0.3"\n      }\n    ]\n  }\n]\n```'}

In [39]:
chat_history


[{'role': 'system',
  'content': 'You are an experienced software engineer. Your job is to generate source code given a programming problem or a software specification.'},
 {'role': 'assistant',
  'content': "Sure! Please provide the programming problem or software specification, and I'll generate the source code for you."},
 {'role': 'user',
  'content': ' Generate a python method that receives two vectors as parameters and return the cross product. Generate only the code of the function without any examples or tests.'},
 {'role': 'assistant',
  'content': '```python\nfrom typing import Sequence, Tuple\n\ndef cross_product(v1: Sequence[float], v2: Sequence[float]) -> Tuple[float, float, float]:\n    """\n    Compute the cross product of two 3‑dimensional vectors.\n\n    Parameters\n    ----------\n    v1 : Sequence[float]\n        First vector, should contain exactly three elements.\n    v2 : Sequence[float]\n        Second vector, should contain exactly three elements.\n\n    Returns

In [40]:
messages_a

[{'role': 'system',
  'content': 'You are an experienced software engineer. Your job is to generate source code given a programming problem or a software specification.'},
 {'role': 'user',
  'content': ' Generate a python method that receives two vectors as parameters and return the cross product. Generate only the code of the function without any examples or tests.'},
 {'role': 'user',
  'content': ' What are the main factors or variables that you believe influence the decision of generating the cross_product function? Estimate an attribution weight to each variable. Label the variables in alphabetical order. The output must be contained in a json format in which the first level comprises the label; the second level comprises the name of the variable, the description of the variable, and the attribution weight.'}]